In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
url_housing = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/California_housing.csv'
data_housing = pd.read_csv(url_housing)

### [0]

In [5]:
base = data_housing.dropna()

### [1]

In [29]:
X_var = list(base.columns[0:4]) + list(base.columns[5:7])

a1 = base[X_var].corr()['Target']
a1 = a1[0:-1]
a1 = a1.abs().sort_values(ascending=False).index
print(", ".join(a1))

MedInc, AveRooms, HouseAge, AveBedrms, AveOccup


### [2]

In [49]:
train = base[base['Xgrp'] == 0]
X_var = list(a1[:-1]) + ['Population']

In [50]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(train[X_var], train['Target'])

In [51]:
test = pd.DataFrame([[4, 4, 37, 1, 1200]], columns=X_var)

esti = model.predict(test)[0]
print(f"{math.floor(esti * 1000) / 1000:.3f}")

2.485


### [3]

In [57]:
test_set = base[base['Xgrp'] == 1]
result = model.predict(test_set[X_var])

In [61]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test_set['Target'], result)
rmse = math.sqrt(mse)

print(f"{math.floor(rmse * 1000) / 1000:.3f}")

0.794


### [4]

In [97]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_var = base.columns[0:-2]
        
norm = StandardScaler().fit_transform(base[X_var])
pca = PCA(n_components=6).fit(norm)

print(f"{math.floor(pca.explained_variance_[2] * 1000) / 1000:.3f}")

1.023


In [146]:
pca.explained_variance_

array([1.92873512, 1.31994845, 1.02393441, 1.00352401, 0.65564414,
       0.06850468])

### [5]

In [189]:
# 산출(fit)된 주성분에 따라 Data를 변환
transformed_data = pca.transform(norm)
transformed_data = pd.DataFrame(transformed_data) # DataFrame으로 변환

In [190]:
top5 = pca.explained_variance_.copy()
top5.sort()
top5_val = top5[-5]

top5_idx = [i for i, v in enumerate(pca.explained_variance_)
              if v >= top5_val]

In [191]:
from sklearn.linear_model import LinearRegression

train_X = transformed_data[top5_idx]
train_y = base['Target']

model = LinearRegression().fit(train_X, train_y)

In [193]:
print(f"{math.floor(model.score(train_X, train_y) * 1000) / 1000:.3f}")

0.492
